In [1]:
import pandas as pd 
import numpy as np


In [2]:
from google.colab import files
uploaded=files.upload()

Saving Book2n.xlsx to Book2n.xlsx


In [3]:
data=pd.read_excel('Book2n.xlsx')

In [4]:
data

,conc (ppm),ad dose(g/L),ph value,Temperature(⁰C),time,absorbance,conc,real conc,removal,%removal
0,10,1.0,6,30,15,0.0229,0.789655,0.789655,29.210345,97.270448
1,10,1.0,6,30,30,0.0153,0.527586,0.527586,29.472414,98.143138
2,10,1.0,6,30,45,0.0152,0.524138,0.524138,29.475862,98.154621
3,10,1.0,6,30,60,0.0140,0.482759,0.482759,29.517241,98.292414
4,10,1.0,6,30,120,0.0110,0.379310,0.379310,29.620690,98.636897
...,...,...,...,...,...,...,...,...,...,...
86,30,1.0,6,40,15,0.1484,5.117241,5.117241,24.882759,82.859586
87,30,1.0,6,40,30,0.1252,4.317241,4.317241,25.682759,85.523586
88,30,1.0,6,40,45,0.0791,2.727586,2.727586,27.272414,90.817138
89,30,1.0,6,40,60,0.0563,1.941379,1.941379,28.058621,93.435207


In [5]:
from sklearn.preprocessing import PowerTransformer

In [6]:
pt = PowerTransformer()


In [8]:
dff=pt.fit_transform(data)

In [54]:
df=data

In [10]:
df

,0,1,2,3,4,5,6,7,8,9
0,-2.774094,-0.101091,-0.016075,-0.080550,-1.266266,-0.586941,-0.454824,-0.461947,0.581451,0.578585
1,-2.774094,-0.101091,-0.016075,-0.080550,-0.526452,-0.775195,-0.807595,-0.807275,0.675453,0.668526
2,-2.774094,-0.101091,-0.016075,-0.080550,-0.007143,-0.777772,-0.812981,-0.812542,0.676705,0.669722
3,-2.774094,-0.101091,-0.016075,-0.080550,0.405159,-0.808907,-0.879419,-0.877505,0.691759,0.684109
4,-2.774094,-0.101091,-0.016075,-0.080550,1.569231,-0.888468,-1.061567,-1.055498,0.729641,0.720295
...,...,...,...,...,...,...,...,...,...,...
86,-0.030050,-0.101091,-0.016075,2.610852,-1.266266,1.119409,1.133473,1.104058,-0.672795,-0.640944
87,-0.030050,-0.101091,-0.016075,2.610852,-0.526452,0.937541,1.024342,0.995608,-0.480045,-0.450744
88,-0.030050,-0.101091,-0.016075,2.610852,-0.007143,0.433301,0.681963,0.656421,-0.046584,-0.027071
89,-0.030050,-0.101091,-0.016075,2.610852,0.405159,0.087017,0.389003,0.367217,0.194357,0.206325


In [27]:
data_cols = ['conc (ppm)', 'ad dose(g/L)', 'ph value', 'Temperature(⁰C)', 'time',
       'absorbance', 'conc', 'real conc', 'removal', '%removal  ']

In [12]:
y=data.iloc[:,-1]
x=data.iloc[:,0:5]

In [60]:
from sklearn.model_selection import train_test_split as tts
Xtrain,Xtest,ytrain,ytest=tts(x,y,test_size=0.3,random_state=42)

In [13]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import Model

from tensorflow.keras.optimizers import Adam

In [14]:
class GAN():
    
    def __init__(self, gan_args):
        [self.batch_size, lr, self.noise_dim,
         self.data_dim, layers_dim] = gan_args

        self.generator = Generator(self.batch_size).\
            build_model(input_shape=(self.noise_dim,), dim=layers_dim, data_dim=self.data_dim)

        self.discriminator = Discriminator(self.batch_size).\
            build_model(input_shape=(self.data_dim,), dim=layers_dim)

        optimizer = Adam(lr, 0.5)

        # Build and compile the discriminator
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.noise_dim,))
        record = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(record)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def get_data_batch(self, train, batch_size, seed=0):
        # # random sampling - some samples will have excessively low or high sampling, but easy to implement
        # np.random.seed(seed)
        # x = train.loc[ np.random.choice(train.index, batch_size) ].values
        # iterate through shuffled indices, so every sample gets covered evenly

        start_i = (batch_size * seed) % len(train)
        stop_i = start_i + batch_size
        shuffle_seed = (batch_size * seed) // len(train)
        np.random.seed(shuffle_seed)
        train_ix = np.random.choice(list(train.index), replace=False, size=len(train))  # wasteful to shuffle every time
        train_ix = list(train_ix) + list(train_ix)  # duplicate to cover ranges past the end of the set
        x = train.loc[train_ix[start_i: stop_i]].values
        return np.reshape(x, (batch_size, -1))
        
    def train(self, data, train_arguments):
        [cache_prefix, epochs, sample_interval] = train_arguments
        
        data_cols = data.columns

        # Adversarial ground truths
        valid = np.ones((self.batch_size, 1))
        fake = np.zeros((self.batch_size, 1))

        for epoch in range(epochs):    
            # ---------------------
            #  Train Discriminator
            # ---------------------
            batch_data = self.get_data_batch(data, self.batch_size)
            noise = tf.random.normal((self.batch_size, self.noise_dim))

            # Generate a batch of new images
            gen_data = self.generator.predict(noise)
    
            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(batch_data, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_data, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
            # ---------------------
            #  Train Generator
            # ---------------------
            noise = tf.random.normal((self.batch_size, self.noise_dim))
            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)
    
            # Plot the progress
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss))
    
            # If at save interval => save generated events
            if epoch % sample_interval == 0:
                #Test here data generation step
                # save model checkpoints
                model_checkpoint_base_name = 'model/' + cache_prefix + '_{}_model_weights_step_{}.h5'
                self.generator.save_weights(model_checkpoint_base_name.format('generator', epoch))
                self.discriminator.save_weights(model_checkpoint_base_name.format('discriminator', epoch))

                #Here is generating the data
                z = tf.random.normal((432, self.noise_dim))
                gen_data = self.generator(z)
                print('generated_data')

    def save(self, path, name):
        assert os.path.isdir(path) == True, \
            "Please provide a valid path. Path must be a directory."
        model_path = os.path.join(path, name)
        self.generator.save_weights(model_path)  # Load the generator
        return
    
    def load(self, path):
        assert os.path.isdir(path) == True, \
            "Please provide a valid path. Path must be a directory."
        self.generator = Generator(self.batch_size)
        self.generator = self.generator.load_weights(path)
        return self.generator
    
class Generator():
    def __init__(self, batch_size):
        self.batch_size=batch_size
        
    def build_model(self, input_shape, dim, data_dim):
        input= Input(shape=input_shape, batch_size=self.batch_size)
        x = Dense(dim, activation='relu')(input)
        x = Dense(dim * 2, activation='relu')(x)
        x = Dense(dim * 4, activation='relu')(x)
        x = Dense(data_dim)(x)
        return Model(inputs=input, outputs=x)

class Discriminator():
    def __init__(self,batch_size):
        self.batch_size=batch_size
    
    def build_model(self, input_shape, dim):
        input = Input(shape=input_shape, batch_size=self.batch_size)
        x = Dense(dim * 4, activation='relu')(input)
        x = Dropout(0.1)(x)
        x = Dense(dim * 2, activation='relu')(x)
        x = Dropout(0.1)(x)
        x = Dense(dim, activation='relu')(x)
        x = Dense(1, activation='sigmoid')(x)
        return Model(inputs=input, outputs=x)

In [15]:
df.shape[1]

10

In [16]:
noise_dim = 32
dim = 128
batch_size = 32

log_step = 100
epochs = 5000+1
learning_rate = 5e-4
models_dir = 'model'



gan_args = [batch_size, learning_rate, noise_dim, df.shape[1], dim]
train_args = ['', epochs, log_step]

In [17]:
!mkdir model

In [55]:
model = GAN

#Training the GAN model chosen: Vanilla GAN, CGAN, DCGAN, etc.
synthesizer = model(gan_args)
synthesizer.train(df,train_args)

Streaming output truncated to the last 5000 lines.
51 [D loss: 0.595485, acc.: 50.00%] [G loss: 0.807551]
52 [D loss: 0.494691, acc.: 51.56%] [G loss: 0.709626]
53 [D loss: 0.455056, acc.: 64.06%] [G loss: 0.796149]
54 [D loss: 0.493709, acc.: 60.94%] [G loss: 0.913466]
55 [D loss: 0.352490, acc.: 87.50%] [G loss: 1.185205]
56 [D loss: 0.400666, acc.: 81.25%] [G loss: 1.138453]
57 [D loss: 0.627782, acc.: 48.44%] [G loss: 0.852632]
58 [D loss: 0.542318, acc.: 50.00%] [G loss: 0.855050]
59 [D loss: 0.573975, acc.: 45.31%] [G loss: 0.881803]
60 [D loss: 0.568620, acc.: 53.12%] [G loss: 0.811717]
61 [D loss: 0.529307, acc.: 50.00%] [G loss: 0.811808]
62 [D loss: 0.495764, acc.: 54.69%] [G loss: 0.851987]
63 [D loss: 0.577664, acc.: 43.75%] [G loss: 0.767389]
64 [D loss: 0.454873, acc.: 59.38%] [G loss: 0.897777]
65 [D loss: 0.370225, acc.: 92.19%] [G loss: 1.063193]
66 [D loss: 0.404121, acc.: 85.94%] [G loss: 1.153931]
67 [D loss: 0.369670, acc.: 92.19%] [G loss: 1.077587]
68 [D loss: 0.

In [19]:
!mkdir model/gan
!mkdir model/gan/saved

In [20]:
synthesizer.generator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(32, 32)]                0         
                                                                 
 dense (Dense)               (32, 128)                 4224      
                                                                 
 dense_1 (Dense)             (32, 256)                 33024     
                                                                 
 dense_2 (Dense)             (32, 512)                 131584    
                                                                 
 dense_3 (Dense)             (32, 10)                  5130      
                                                                 
Total params: 173,962
Trainable params: 173,962
Non-trainable params: 0
_________________________________________________________________


In [21]:
synthesizer.discriminator.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(32, 10)]                0         
                                                                 
 dense_4 (Dense)             (32, 512)                 5632      
                                                                 
 dropout (Dropout)           (32, 512)                 0         
                                                                 
 dense_5 (Dense)             (32, 256)                 131328    
                                                                 
 dropout_1 (Dropout)         (32, 256)                 0         
                                                                 
 dense_6 (Dense)             (32, 128)                 32896     
                                                                 
 dense_7 (Dense)             (32, 1)                   129 

In [22]:
models = {'GAN': ['GAN', False, synthesizer.generator]}

In [56]:
import matplotlib.pyplot as plt

# Setup parameters visualization parameters
seed = 17
test_size = 151
noise_dim = 32

np.random.seed(seed)
z = np.random.normal(size=(test_size, noise_dim))
real = synthesizer.get_data_batch(train=df, batch_size=test_size, seed=seed)
real_samples = pd.DataFrame(real, columns=data_cols)

In [40]:
model_names = ['GAN']
colors = ['deepskyblue','blue']
markers = ['o','^']

In [41]:
data_cols

['conc (ppm)',
 'ad dose(g/L)',
 'ph value',
 'Temperature(⁰C)',
 'time',
 'absorbance',
 'conc',
 'real conc',
 'removal',
 '%removal  ']

In [42]:
col1,col2,col3,col4,col5,col6,col7,col8,col9,col10='conc (ppm)','ad dose(g/L)','ph value','Temperature(⁰C)','time','absorbance','conc','real conc','removal','%removal  '

In [43]:
base_dir = 'model/'

In [48]:
model_steps= [ 0, 100, 200, 300, 400, 500, 1000, 2000, 3000, 4000, 5000]
rows = len(model_steps)
columns = 10


In [57]:

for model_step_ix, model_step in enumerate(model_steps):
  [model_name, with_class, generator_model] = models['GAN']

  generator_model.load_weights( base_dir + '_generator_model_weights_step_'+str(model_step)+'.h5')

  #ax = plt.subplot(rows, columns, model_step_ix*columns + 1 + (i+1) )

  g_z = generator_model.predict(z)
  gen_samples = pd.DataFrame(g_z, columns=data_cols)
  gen_samples.to_csv('Generated_sample.csv')

In [68]:
yy=gen_samples.iloc[:,-1]
xx=gen_samples.iloc[:,0:5]

In [59]:
 gen_samples

,conc (ppm),ad dose(g/L),ph value,Temperature(⁰C),time,absorbance,conc,real conc,removal,%removal
0,27.660261,1.618244,3.735134,27.909702,13.476619,1.805746,2.650773,2.352627,26.605284,89.788040
1,26.326616,3.667342,5.331314,27.241755,48.583408,-0.548532,4.777030,0.629774,25.725002,89.138596
2,22.852629,1.004803,3.059339,20.674023,14.333040,1.631472,2.444756,1.683973,20.907017,67.832657
3,26.600159,2.383255,4.866921,25.024033,34.677898,0.359784,3.459568,0.788936,25.646530,85.929886
4,22.013552,3.259620,4.223422,23.429605,45.440678,-0.943759,4.218081,0.578856,21.645710,76.650772
...,...,...,...,...,...,...,...,...,...,...
146,18.962521,2.272252,3.651122,20.312044,37.573811,-0.082875,3.364410,0.082747,18.730110,65.727425
147,15.966755,3.987911,4.847888,28.787365,56.483368,2.084304,3.031299,-0.948794,29.215036,102.097305
148,24.133488,2.870627,4.637295,28.265459,21.723787,2.071230,3.212455,1.167626,28.457733,100.045425
149,30.119204,0.792066,3.581024,28.310966,11.824623,2.167235,2.864714,3.531495,27.194139,87.186119


In [61]:
pip install keras-tuner

     |████████████████████████████████| 98 kB 2.3 MB/s 


In [62]:
import kerastuner
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [63]:
def model(hp):
  build_model=Sequential()
  build_model.add(Dense(units=hp.Int('n',min_value=16,max_value=512,step=16),activation='relu',input_dim=Xtrain.shape[1]))
  for i in range(hp.Int('nlayers',min_value=2,max_value=10)):
    build_model.add(Dense(units=hp.Int('units',min_value=16,max_value=256,step=16),activation='relu'))
  build_model.add(Dense(units=1,activation='linear'))
  build_model.compile(optimizer=Adam(hp.Choice('lr',[1e-2,1e-3,1e-4])),loss='mean_absolute_error',metrics=['mean_absolute_error'])
  return build_model

In [64]:
tuner=RandomSearch(model,objective='val_mean_absolute_error',max_trials=5,executions_per_trial=3,directory='output',project_name='chemann')

In [66]:
tuner.search(Xtrain,ytrain,epochs=20,validation_data=(Xtest,ytest),batch_size=2)

Trial 6 Complete [00h 00m 15s]
val_mean_absolute_error: 11.22193972269694

Best val_mean_absolute_error So Far: 9.41278330485026
Total elapsed time: 00h 01m 44s
INFO:tensorflow:Oracle triggered exit


In [67]:
best_model=tuner.get_best_models(num_models=1)[0]

In [69]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 112)               672       
                                                                 
 dense_1 (Dense)             (None, 80)                9040      
                                                                 
 dense_2 (Dense)             (None, 80)                6480      
                                                                 
 dense_3 (Dense)             (None, 80)                6480      
                                                                 
 dense_4 (Dense)             (None, 80)                6480      
                                                                 
 dense_5 (Dense)             (None, 80)                6480      
                                                                 
 dense_6 (Dense)             (None, 80)                6

In [70]:
ypred=best_model.predict(xx)

In [71]:
yy

0       89.788040
1       89.138596
2       67.832657
3       85.929886
4       76.650772
          ...    
146     65.727425
147    102.097305
148    100.045425
149     87.186119
150     90.159698
Name: %removal  , Length: 151, dtype: float32

In [72]:
ypred

array([[ 84.86184 ],
       [ 93.05983 ],
       [ 65.6334  ],
       [ 83.50088 ],
       [ 80.93003 ],
       [ 64.63751 ],
       [ 60.263176],
       [111.1159  ],
       [103.04861 ],
       [ 62.250374],
       [ 76.94509 ],
       [ 60.11001 ],
       [ 63.940857],
       [ 51.90059 ],
       [ 64.127144],
       [112.61979 ],
       [ 71.270546],
       [ 74.048485],
       [ 93.56954 ],
       [ 81.65387 ],
       [ 64.00303 ],
       [107.53633 ],
       [ 65.40314 ],
       [ 70.52867 ],
       [145.94589 ],
       [ 65.83022 ],
       [ 56.967422],
       [ 94.36686 ],
       [ 71.44091 ],
       [ 80.33999 ],
       [ 53.30298 ],
       [ 35.18811 ],
       [ 83.58815 ],
       [ 53.949833],
       [132.50511 ],
       [ 54.250637],
       [ 72.49631 ],
       [ 75.63415 ],
       [128.55566 ],
       [111.81977 ],
       [ 62.929775],
       [ 75.136116],
       [ 64.22817 ],
       [115.15734 ],
       [100.777435],
       [ 81.36262 ],
       [ 58.47236 ],
       [ 52.7